In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Alopecia/GSE81071'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression from human discoid (DLE) and subacute (sCLE) cutaneous lupus subtypes"
!Series_summary	"Cutaneous lupus erythematosus (CLE) is a disfiguring disease that can exist as an independent entity or as a manifestation of systemic lupus erythematosus (SLE) where up to 70% of patients experience lesions during their disease course.  Subacute CLE (sCLE) is an inflammatory lesion with associated erythema in papulosquamous or annular formations.  Typically, sCLE does not scar but depigmentation can occur. Importantly, sCLE is associated with a higher progression to SLE. Discoid lesions (DLE) are often circular and frequently lead to alopecia and scar formation.  sCLE lesions have a higher propensity for photoprovocation and a more robust inflammatory infiltrate following ultraviolet (UV) B exposure. The pathogenic mechanisms which govern the differences between DLE and sCLE remain poorly defined, and this is reflected by the refractory nature 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True

# Determine availability of specific characteristics
characteristics_dict = {
    0: ['tissue: Skin', 'disease state: Normal', 'disease state: DLE', 'disease state: SCLE'], 
    1: ['disease state: healthy', 'disease state: DLE', 'disease state: sCLE', 'tissue: Skin biopsy']
}

# trait_row determination
if any('disease state: DLE' in value or 'disease state: SCLE' in value for values in characteristics_dict.values() for value in values):
    trait_row = next((key for key, values in characteristics_dict.items() if any('disease state: DLE' in value or 'disease state: SCLE' in value for value in values)), None)
else:
    trait_row = None

# We didn't find explicit references to age or gender in the characteristics:
age_row, gender_row = None, None

# Define conversion functions
def convert_trait(value):
    val = value.split(':')[-1].strip().lower()
    if 'dle' in val:
        return 1
    elif 'scle' in val:
        return 0
    return None

def convert_age(value):
    val = value.split(':')[-1].strip()
    try:
        return float(val)
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(':')[-1].strip().lower()
    if val == 'male':
        return 1
    elif val == 'female':
        return 0
    return None

# Save cohort information
save_cohort_info('GSE81071', './preprocessed/Alopecia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction step
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Alopecia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Alopecia/trait_data/GSE81071.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2142137': [None], 'GSM2142138': [None], 'GSM2142139': [None], 'GSM2142140': [None], 'GSM2142141': [None], 'GSM2142142': [None], 'GSM2142143': [None], 'GSM2142144': [None], 'GSM2142145': [None], 'GSM2142146': [None], 'GSM2142147': [None], 'GSM2142148': [None], 'GSM2142149': [None], 'GSM2142150': [None], 'GSM2142151': [None], 'GSM2142152': [None], 'GSM2142153': [None], 'GSM2142154': [None], 'GSM2142155': [None], 'GSM2142156': [None], 'GSM2142157': [None], 'GSM2142158': [None], 'GSM2142159': [None], 'GSM2142160': [None], 'GSM2142161': [None], 'GSM2142162': [None], 'GSM2142163': [None], 'GSM2142164': [None], 'GSM2142165': [None], 'GSM2142166': [None], 'GSM2142167': [None], 'GSM2142168': [None], 'GSM2142169': [None], 'GSM2142170': [None], 'GSM2142171': [None], 'GSM2142172': [None], 'GSM2142173': [None], 'GSM2142174': [None], 'GSM2142175': [None], 'GSM2142176': [None], 'GSM2142177': [None], 'GSM2142178': [None], 'GSM2142179': [None], 'GSM2142180': [None], 'GSM2142181': [None], 'GSM21421

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033413_at', '100033422_at', '100033423_at', '100033424_at',
       '100033425_at', '100033426_at', '100033427_at', '100033428_at',
       '100033430_at', '100033431_at', '100033432_at', '100033434_at',
       '100033435_at', '100033436_at', '100033437_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ENTREZ_GENE_ID': ['1', '10', '100', '1000', '10000']}


### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Reading dictionary keys
    identifier_key = 'ID'
    gene_symbol_key = 'ENTREZ_GENE_ID'
    
    # 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
    gene_mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    
    # 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data".
    gene_data = apply_gene_mapping(gene_data, gene_mapping_df)
